In [7]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd


In [3]:
data = pd.read_csv("./diabetic_data.csv")
data.replace('?',np.nan,inplace=True)

In [63]:
from math import log10
class ExploratoryDataAnalysis:
    def __init__(self,data):
        self.data = data
    
    def get_feature_class_count(self,col):
        count_map = {}
        for x in col:
            if(x in count_map):
                count_map[x] = count_map[x] + 1
            else:
                count_map[x] = 1

        return log10(len(count_map))
    
    def get_missing_counts(self,col):
        return len(col) - col.count()
    
    def plot_class_counts(self,plot_missing_feature_count = False):
        class_counts = [self.get_feature_class_count(self.data[column]) for column in self.data]
        data = [go.Histogram(x=self.data.columns, y=class_counts)]
        trace = go.Bar(
            x=self.data.columns,
            y=class_counts
        )
        layout = go.Layout(
            title='Feature counts | Total number of rows=' + str(len(self.data)),
            xaxis = dict(title='Features'),
            yaxis = dict(title='Log(Number of unique values)')
        )
        py.iplot(go.Figure(data=[trace], layout=layout))
    
    def plot_class_count(self):
        count = 0
        for x in self.data["readmitted"]:
            if(x == "NO"):
                count = count + 1

        trace = go.Bar(
            x=["YES","NO"],
            y=[(len(self.data) - count),count]
        )
        layout = go.Layout(
            title='Class counts | Total number of rows=' + str(len(self.data)),
            xaxis = dict(title='Features'),
            yaxis = dict(title='Log(Number of unique values)')
        )
        py.iplot(go.Figure(data=[trace], layout=layout))
        
    def plot_missing_count(self):
        class_counts = [len(self.data[column]) for column in self.data]
        missing_counts = [self.get_missing_counts(self.data[column]) for column in self.data]
        data = [go.Histogram(x=self.data.columns, y=class_counts)]
        trace = go.Bar(
            x=self.data.columns,
            y=class_counts
        )
        trace_missing = go.Bar(
            x=self.data.columns,
            y=missing_counts
        )
        layout = go.Layout(
            title='Feature counts | Total number of rows=' + str(len(self.data)),
            xaxis = dict(title='Features'),
            yaxis = dict(title='Log(Number of unique values)')
        )
        py.iplot(go.Figure(data=[trace,trace_missing], layout=layout))
        

    

In [64]:
EDA = ExploratoryDataAnalysis(data)
EDA.plot_class_counts()
EDA.plot_class_count()
EDA.plot_missing_count()

In [212]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack,vstack

class DummyOneHotEncoder:
    def __init__(self,threshold_count=20,dummy_variable="UNK"):
        self.enc = OneHotEncoder(handle_unknown='ignore',sparse=True,categories='auto')
        self.frequency_map = {}
        self.threshold_count = threshold_count
        self.dummy_variable = dummy_variable
    
    def fit(self,data):
        map(self._process_frequencies,enumerate(data))
        map(self._process_rare_data,enumerate(data))
        return self.enc.fit(data)
        
    def _process_frequencies(self,data,index):
        if(data[index] not in self.frequency_map):
            self.frequency_map[data[index]] = 1
        else:
            self.frequency_map[data[index]] = self.frequency_map[data[index]] + 1
            
    def _process_rare_data(self,data,index):
        if(self.frequency_map[data[index]] <= self.threshold_count):
            data[index] = self.dummy_variable
            
    def fit_transform(self,data):
        map(self._process_frequencies,enumerate(data))
        map(self._process_rare_data,enumerate(data))
        return self.enc.fit_transform(data)
    
    def transform(self,data):
        map(self._process_rare_data,enumerate(data))
        return self.enc.transform(data)
            
    def get_encoder(self):
        return self.enc
    
class DummyTextEncoder:
    def __init__(self,threshold_count=20,dummy_variable="UNK"):
        self.enc = CountVectorizer()
        self.frequency_map = {}
        self.threshold_count = threshold_count
        
        
    def fit(self,data):
        map(self._process_frequencies,enumerate(data))
        map(self._process_rare_data,enumerate(data))
        return self.enc.fit(data)
        
    def _process_frequencies(self,data,index):
        if(data[index] not in self.frequency_map):
            self.frequency_map[data[index]] = 1
        else:
            self.frequency_map[data[index]] = self.frequency_map[data[index]] + 1
            
    def _process_rare_data(self,data,index):
        if(self.frequency_map[data[index]] <= self.threshold_count):
            data[index] = self.dummy_variable
            
    def fit_transform(self,data):
        map(self._process_frequencies,enumerate(data))
        map(self._process_rare_data,enumerate(data))
        return self.enc.fit_transform(data)
    
    def transform(self,data):
        map(self._process_rare_data,enumerate(data))
        return self.enc.transform(data)
            
    def get_encoder(self):
        return self.enc


class CategoricalEncoder:
    def __init__(self):
        self.label_encoder = LabelEncoder()
        self.one_hot_encoder = DummyOneHotEncoder()
       
    def fit(self,data):
        labels = self.label_encoder.fit(data)
        return self.one_hot_encoder.fit([labels])
   
    def fit_transform(self,data):
        labels = self.label_encoder.fit_transform(data)
        labels = labels.reshape(-1,1)
        encoded_data = self.one_hot_encoder.fit_transform(labels)
        return encoded_data
   
    def transform(self,data):
        labels = self.label_encoder.transform(data)
        return self.one_hot_encoder.transform([labels])
    
class FeatureEngineering:
       
    def get_encoding(self,column):
        categorical_encoder = CategoricalEncoder()
        return categorical_encoder.fit_transform(column.astype('str')), categorical_encoder
    
    def fit(self,data,y):
        self.categorical_indices={
            "gender": True,
            "age":True,
            "admission_type_id":True,
            "discharge_disposition_id":True,
            "admission_source_id": True,
            "time_in_hospital": False,
            "medical_specialty":True,
            "num_lab_procedures":False,
            "num_procedures":False,
            "num_medications":False,
            "number_outpatient":True,
            "number_emergency":True,
            "number_inpatient":True,
            "diag_1":True,
            "diag_2":True,
            "diag_3":True,
            "number_diagnoses":False,
            "max_glu_serum":True,
            "A1Cresult":True,
            "metformin":True,
            "repaglinide":True,
            "nateglinide":True,
            "chlorpropamide":True,
            "glimepiride":True,
            "acetohexamide":True,
            "glipizide":True,
            "glyburide":True,
            "tolbutamide":True,
            "pioglitazone":True,
            "rosiglitazone":True,
            "acarbose":True,
            "miglitol":True,
            "troglitazone":True,
            "tolazamide":True,
            "examide":True,
            "citoglipton":True,
            "insulin":True,
            "glyburide-metformin":True,
            "glipizide-metformin":True,
            "metformin-rosiglitazone":True,
            "metformin-pioglitazone":True,
            "change":True,
            "diabetesMed":True
        }
        x = None
        self.encoder = {}
        for label, column in data.items():
            if(label in self.categorical_indices):
                if(self.categorical_indices[label] == True):
                    encoded_data,self.encoder[label] = self.get_encoding(column)
                    x = encoded_data if x == None else hstack([x,encoded_data],format="csr")
                else:
                    x = column if x == None else hstack([x,((pd.DataFrame(column.astype('float64'))).to_sparse())],format="csr")
        return x
    
    def fit_transform(self,data,y):
        return self.fit(data,y)
    
    def transform(self,data):
        x = None
        for label, column in data.items():
            if(label in self.categorical_indices):
                if(self.categorical_indices[label] == True):
                    encoded_data = (self.encoder[label]).transform(column)
                    x = encoded_data if x == None else hstack([x,encoded_data])
                else:
                    x = column if x == None else hstack([x,(pd.DataFrame(column).to_sparse())])
        return x
    

In [213]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import train_test_split

def split_data(x,label):
    X_train, X_test, y_train, y_test = train_test_split(x, label, test_size=0.10)
    return X_train, X_test, y_train, y_test

y = (data["readmitted"].astype('str')).apply(lambda x: 0 if x == "NO" else (1 if x == "<30" else 2))
X_train, X_test, y_train, y_test = split_data(data,y) 
pipeline = make_pipeline(FeatureEngineering(),LogisticRegression())
pipeline.fit(X_train,y_train)
pipeline.score(X_test,y_test)


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



IndexError: list index out of range